This notebook allow you to print results of models in tables

In [1]:
import os ; os.environ["CUDA_VISIBLE_DEVICES"] = ""

In [2]:
from systemtools.hayj import *
from systemtools.location import *
from systemtools.basics import *
from systemtools.file import *
from systemtools.printer import *
from twinews.utils import *
from twinews.evaluation.utils import *

In [3]:
if False:
    purgeSubsampledRankings()

In [4]:
def basicPrint(model, **kwargs):
    printReportKwargs = \
    {
        'model': model,
        'splitVersion': 2,
        'noSubsampling': True,
        'metaFilter': \
        {
            # "implementation": "gensim-lda",
        },
        'metricsFilter': {'ndcg'}, # None, {'ndcg', 'ndcg@10'}
        'sortBy': 'ndcg',
    }
    printReportKwargs = mergeDicts(printReportKwargs, kwargs)
    printReport(**printReportKwargs)

# Reference models

In [5]:
splitVersion = 2
df = printReport(model='ideal', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')
df = printReport(model='random', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')
df = printReport(model='worst', splitVersion=splitVersion, noSubsampling=True, sortBy='ndcg')

,id,maxUsers,model,splitVersion,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
0,ideal-5ac2d,None,ideal,2,1,1,1,1,1,0.56539,0.07227


These values are common to all rows:

	- splitVersion: 2
	- model: random
	- maxUsers: None


,id,seed,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
0,random-f03b8,4,0.0109,0.01815,0.22153,0.00019,0.05189,0.0003,0.01156
2,random-4d05f,8,0.0096,0.01258,0.21667,0.00282,0.0034,0.00442,0.00137
9,random-6fae4,3,0.00979,0.00786,0.21645,0,0.00067,0,0.00042
8,random-04de9,2,0.00869,0.0086,0.21099,0,0.00098,0,0.00048
5,random-a9655,1,0.0083,0.00934,0.21023,0.00027,0.02439,0.00041,0.00688
3,random-21e0b,6,0.00871,0.02293,0.20917,0.01098,0.00966,0.01582,0.00261
1,random-d87ed,9,0.00773,0.00606,0.20841,0,0.00015,0,0.00012
4,random-a1ece,7,0.00828,0.01206,0.20738,0,0.0069,0,0.00195
6,random-093c0,0,0.00783,0.01652,0.20427,0,0.01671,0,0.00396
7,random-e27ab,5,0.00738,0.00755,0.20295,0.0002,0.0172,0.00028,0.00583


,id,maxUsers,model,splitVersion,map,mrr,ndcg,ndcg@10,ndcg@100,p@10,p@100
0,worst-f3f99,None,worst,2,0.00413,0.001,0.18191,0,0,0,0


# Ad-hoc models

In [ ]:
basicPrint('bm25')

In [ ]:
basicPrint('BM25_yf', noSubsampling=False)

# Models that use vector representations

In [ ]:
basicPrint('lda')

In [ ]:
basicPrint('nmf')

In [ ]:
basicPrint('tfidf')

In [ ]:
basicPrint('dbert-ft')

In [ ]:
basicPrint('dbert-base')

In [ ]:
basicPrint('stylo')

In [ ]:
basicPrint('infersent')

In [ ]:
basicPrint('sent2vec')

In [ ]:
basicPrint('word2vec')

In [ ]:
basicPrint('doc2vec')

In [ ]:
basicPrint('usent')

In [ ]:
basicPrint('bert')

# Combinations of models

In [8]:
basicPrint('combin', noSubsampling=False)

These values are common to all rows:

	- rankAsScore: [True, True]
	- weights: [0.5, 0.5]
	- splitVersion: 2
	- alphas: [0.5, 0.5]
	- betas: ['LOG', 'LOG']
	- model: combin


,id,maxUsers,models,ndcg
1,combin-94b4a,2,"['bert-65e2b', 'bm25-1eb2a']",0.39292
0,combin-aba93,2,"['bert-65e2b', 'dbert-base-aafd1']",0.3304
2,combin-e8ef3,nan,"['bert-65e2b', 'bm25-1eb2a']",nan


# Deep Learning models